In [17]:
import pygame
import random
import time
import sys

pygame.init()


WIDTH, HEIGHT = 600, 600
ROWS, COLS = 4, 4
TILE_SIZE = WIDTH // COLS
FPS = 30
TIME_LIMIT = 30

# Colors
WHITE = (255, 255, 255)
GRAY = (100, 100, 100)
BLACK = (0, 0, 0)
BLUE = (50, 50, 200)
GREEN = (0, 200, 0)
RED = (200, 0, 0)


screen = pygame.display.set_mode((WIDTH, HEIGHT + 50))
pygame.display.set_caption("Memory Puzzle Game")
font = pygame.font.SysFont(None, 40)
clock = pygame.time.Clock()


symbols = list(range((ROWS * COLS) // 2)) * 2
random.shuffle(symbols)
board = [symbols[i * COLS:(i + 1) * COLS] for i in range(ROWS)]
revealed = [[False] * COLS for _ in range(ROWS)]

first_selection = None
start_time = time.time()


def draw_tile(row, col, value, show):
    x = col * TILE_SIZE
    y = row * TILE_SIZE
    rect = pygame.Rect(x, y, TILE_SIZE, TILE_SIZE)
    pygame.draw.rect(screen, BLUE if show else GRAY, rect)
    pygame.draw.rect(screen, WHITE, rect, 2)
    if show:
        text = font.render(str(value), True, WHITE)
        text_rect = text.get_rect(center=rect.center)
        screen.blit(text, text_rect)


def main():
    global first_selection
    matched_pairs = 0
    running = True

    while running:
        screen.fill(BLACK)

        elapsed_time = int(time.time() - start_time)
        remaining_time = max(0, TIME_LIMIT - elapsed_time)

        
        timer_text = font.render(f"Time Left: {remaining_time}s", True, RED)
        screen.blit(timer_text, (10, HEIGHT + 10))

        if remaining_time == 0:
            game_over("⏰ Time's up! You lost.")
            return

    
        for r in range(ROWS):
            for c in range(COLS):
                val = board[r][c]
                draw_tile(r, c, val, revealed[r][c])

        pygame.display.flip()
        clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                safe_exit()

            elif event.type == pygame.MOUSEBUTTONDOWN:
                x, y = event.pos
                if y > HEIGHT:
                    continue  
                col = x // TILE_SIZE
                row = y // TILE_SIZE

                if not revealed[row][col]:
                    revealed[row][col] = True
                    if first_selection is None:
                        first_selection = (row, col)
                    else:
                        r1, c1 = first_selection
                        r2, c2 = row, col

                        if board[r1][c1] != board[r2][c2]:
                            pygame.display.flip()
                            pygame.time.delay(1000)
                            revealed[r1][c1] = False
                            revealed[r2][c2] = False
                        else:
                            matched_pairs += 1

                        first_selection = None

        if matched_pairs == (ROWS * COLS) // 2:
            game_over("🎉 Congratulations! You won!")
            return

def game_over(message):
    screen.fill(BLACK)
    text = font.render(message, True, GREEN if "won" in message else RED)
    text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
    screen.blit(text, text_rect)
    pygame.display.flip()
    pygame.time.delay(3000)
    safe_exit()


def safe_exit():
    pygame.quit()
    try:
        sys.exit()
    except SystemExit:
        pass  


if __name__ == "__main__":
    main()
